In [1]:
import numpy as np

In [2]:
import apple

In [3]:
train_images=np.load('data/train-images.npy')

In [4]:
reshape=apple.jit("[♭`{3∘[2,3,4]} (x::Arr (60000 × 28 × 28 × 1) float)]")

In [5]:
train_images_v=apple.f(reshape,train_images)

In [6]:
train_labels=np.load('data/train-labels.npy')

In [7]:
def sigmoid(x):
    return 1/(np.exp(-x)+1)

In [8]:
def d_sigmoid(x):
    return (np.exp(-x))/((np.exp(-x)+1)**2)

In [9]:
def softmax(x):
    exp_element=np.exp(x-x.max())
    return exp_element/np.sum(exp_element,axis=0)

In [10]:
def d_softmax(x):
    exp_element=np.exp(x-x.max())
    return exp_element/np.sum(exp_element,axis=0)*(1-exp_element/np.sum(exp_element,axis=0))

In [11]:
def init(x,y):
    return np.random.uniform(-1.,1.,(x,y))/np.sqrt(x*y)

In [12]:
np.random.seed(17)

In [13]:
l1=init(28*28,128)

In [14]:
l2=init(128,10)

In [15]:
ssoftmax=apple.jit('''
λxs.
  { m ⟜ (⋉)/* _1 xs; a ⟜ [e:(x-m)]`{0} xs
  ; sum ← [(+)/x]
  ; n ⟜ sum`{1} (a::M float)
  ; ⍉(([(%x)'y]`{0,1} n a))
  }
''')

In [16]:
vsigmoid=apple.jit("([1%(1+ℯ(_x))]`{0})")

In [17]:
vmap=apple.jit("((λn.[?x=n,.1::float,.0]'irange 0 9 1)')")

In [18]:
mmul=apple.jit("[x%.(y::M float)]")

In [19]:
train_labels_v=apple.f(vmap,train_labels)

In [20]:
def fw_py(l1,l2,x):
    x_l1p=x.dot(l1)
    x_sigmoid=sigmoid(x_l1p)
    x_l2p=x_sigmoid.dot(l2)
    out=softmax(x_l2p)
    return x_l1p,x_sigmoid,x_l2p,out

In [21]:
def fw(l1,l2,x):
    x_l1p=apple.f(mmul,x,l1)
    x_sigmoid=apple.f(vsigmoid,x_l1p)
    x_l2p=apple.f(mmul,x_sigmoid,l2)
    out=apple.f(ssoftmax,x_l2p)
    return x_l1p,x_sigmoid,x_l2p,out

In [22]:
errorjit=apple.jit('''
λout.λtargets.λxl2p.
{
  dsoftmax ← λxs.
    { m ⟜ (⋉)/* _1 xs; a ⟜ [e:(x-m)]`{0} xs
    ; sum ← [(+)/x]
    ; n ⟜ sum`{1} (a::M float)
    ; ⍉([x*(1-x)]`{0} ([(%x)'y]`{0,1} n a))
    };
  (*)`{0,0} ({n⟜ℝ(𝓉out); [2*x%n]`{0} ((-)`{0,0} out targets)}) (dsoftmax xl2p)
}
''')

In [23]:
u_l2jit=apple.jit('[(|:(x::M float))%.y]')

In [24]:
u_l1jit=apple.jit('''
λx.λl2.λerror.λxl1p.
{
  dsigmoid ← ((λx.⸎x⟜ℯ(_x);x%(1+x)^2)`{0});
  (⍉x)%.((*)`{0,0} (⍉(l2%.(⍉error))) (dsigmoid xl1p))
}
''')

In [25]:
def fw_bw_py(x,y):
    targets = np.zeros((len(y),10), np.float64)
    targets[range(targets.shape[0]),y] = 1
    
    x_l1p,x_sigmoid,x_l2p,out=fw_py(l1,l2,x)
    
    error=2*(out-targets)/out.shape[0]*d_softmax(x_l2p)
    update_l2=x_sigmoid.T@error
    
    error=((l2).dot(error.T)).T*d_sigmoid(x_l1p)
    update_l1=x.T@error
    
    return out,update_l1,update_l2

In [26]:
def fw_bw(x,targets):
    
    x_l1p,x_sigmoid,x_l2p,out=fw(l1,l2,x)


    error=apple.f(errorjit,out,targets,x_l2p)
    
    update_l2=apple.f(u_l2jit,x_sigmoid,error)
    update_l1=apple.f(u_l1jit,x,l2,error,x_l1p)

    return out,update_l1,update_l2

In [30]:
%timeit fw_bw(train_images_v,train_labels_v)

1min ± 2.96 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%timeit fw_bw_py(train_images_v,train_labels)

385 ms ± 5.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
